# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data['SalesID'] = data['SalesID'].astype(str)
data['ProductID'] = data['ProductID'].astype(str)

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
df = data.groupby(['CustomerID', 'ProductName']).sum()
df.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
prod_cust = pd.pivot_table(data=df, index='ProductName', columns='CustomerID', values='Quantity', aggfunc=sum).fillna(0)
prod_cust.head()
prod_cust.shape

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
cust_similarity = squareform(pdist(prod_cust.T, 'euclidean'))
cust_similarity

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
customer = 33
distances = pd.DataFrame(1/(1 + cust_similarity), 
                         index=prod_cust.columns, columns=prod_cust.columns)

top5 = distances[customer].sort_values(ascending=False)[1:6]
top5

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
top5_purchases = df.loc[list(top5.index)]
# top5_purchases

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
top5_products = top5_purchases.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
top5_products.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
customer_prods = prod_cust.loc[:, customer]
customer_prods = list(customer_prods[customer_prods!=0].index)
# series del step 6 filtrada por productos que el customer no ha comprado nunca
not_purchased = list(top5_products.loc[~top5_products.index.isin(customer_prods)][:5].index)
not_purchased

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [52]:
recommendations = {}
customers = list(set(data['CustomerID']))
for customer in customers:
    top5 = distances[customer].sort_values(ascending=False)[1:6]
    top5_purchases = df.loc[list(top5.index)]
    top5_products = top5_purchases.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
    customer_prods = prod_cust.loc[:, customer]
    customer_prods = list(customer_prods[customer_prods!=0].index)
    not_purchased = list(top5_products.loc[~top5_products.index.isin(customer_prods)][:5].index)
    recommendations[customer] = not_purchased

AttributeError: 'dict' object has no attribute 'head'

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [18]:
dfrecommendations = pd.DataFrame(recommendations).T.reset_index()
cols = ['CustomerID', '0', '1', '2', '3', '4']
dfrecommendations.columns = cols
dfrecommendations.head()

,CustomerID,0,1,2,3,4
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
2,264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
3,356,"Veal - Inside, Choice",Lamb - Ground,Pomello,Butter - Unsalted,"Tart Shells - Sweet, 4"
4,412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [44]:
def recommendations(data, distance_type):
    recommendations = {}
    data['SalesID'] = data['SalesID'].astype(str)
    data['ProductID'] = data['ProductID'].astype(str)
    df = data.groupby(['CustomerID', 'ProductName']).sum()
    prod_cust = pd.pivot_table(data=df, index='ProductName', columns='CustomerID', values='Quantity', aggfunc=sum).fillna(0)
    cust_similarity = squareform(pdist(prod_cust.T, distance_type))
    distances = pd.DataFrame(1/(1 + cust_similarity), index=prod_cust.columns, columns=prod_cust.columns)
    customers = list(set(data['CustomerID']))
    for customer in customers:
        top5 = distances[customer].sort_values(ascending=False)[1:6]
        top5_purchases = df.loc[list(top5.index)]
        top5_products = top5_purchases.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
        customer_prods = prod_cust.loc[:, customer]
        customer_prods = list(customer_prods[customer_prods!=0].index)
        not_purchased = list(top5_products.loc[~top5_products.index.isin(customer_prods)][:5].index)
        recommendations[customer] = not_purchased
    dfrecommendations = pd.DataFrame(recommendations).T.reset_index()
    return dfrecommendations

In [51]:
distance_types = ['euclidean', 'correlation', 'cityblock', 'cosine', 'jaccard']
for distancetype in distance_types:
    print(distancetype)
    display(recommendations(data, distancetype)[:1])

euclidean


,index,0,1,2,3,4
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base


correlation


,index,0,1,2,3,4
0,33,Knife Plastic - White,Muffin - Zero Transfat,Banana Turning,Crush - Cream Soda,Veal - Osso Bucco


cityblock


,index,0,1,2,3,4
0,33,Chocolate - Dark,Wine - Charddonnay Errazuriz,Bay Leaf,"Oranges - Navel, 72",Wine - Ej Gallo Sierra Valley


cosine


,index,0,1,2,3,4
0,33,Knife Plastic - White,"Soup - Campbells, Beef Barley",Onions - Cippolini,Tea - Herbal Sweet Dreams,Muffin - Zero Transfat


jaccard


,index,0,1,2,3,4
0,33,Loquat,Milk - 2%,Muffin Mix - Blueberry,Blueberries,"Cheese - Brie, Triple Creme"
